# Leica .lif Image Processing & Droplet Analysis – Code Documentation



This document explains the code used for processing .lif microscopy files, creating annotated time-lapse GIFs, and performing colocalization / droplet segmentation analysis.


In [ ]:
# Cleaned and organized script for .lif file image processing and threshold analysis

# General
import os, re, glob
import numpy as np
import pandas as pd

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from PIL import Image, ImageDraw, ImageFont

# Image Processing
from skimage.filters import threshold_otsu, threshold_local, gaussian
from skimage.restoration import rolling_ball
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, closing, disk
from scipy.ndimage import binary_fill_holes
from scipy.stats import pearsonr

# File Reader
from readlif.reader import LifFile



# === Visualization Settings ===
colors = ["#56B4E9", "#009E73", "#CC79A7", "#999999", "#E69F00", "#DB2B39", "#0076A1", "#0072B2", "#1A5042", "#0C1713"]
sns.set_theme(context='notebook', style='ticks', font='Arial', font_scale=1.3,
              rc={"lines.linewidth": 1., 'axes.linewidth':1., "xtick.major.width":1.,"ytick.major.width":1.}, 
              palette=sns.color_palette(colors))


# === Image Processing: Kinetics ===


def process_multiple_file_kinetics_merge_max(leica_file, n_samples):



    """


        •	Load .lif files and extract frames from n_samples.
        •	Create average-intensity z-projections. Only one channel (c=0) is used.
        •	Subtract background using rolling-ball algorithm.
        •	Annotate frames with elapsed time and scale bar.
        •	Save animated GIF with all frames.


    """



    file = LifFile(leica_file)
    original_dir = os.getcwd()

    match = re.search(r'(\d+)min(\d+)sec', leica_file)
    total_seconds = int(match.group(1)) * 60 + int(match.group(2)) if match else 0

    files, frames = [], []
    for i in range(n_samples):
        f = file.get_image(i)
        files.append(f)
        frames.append(f.nt)

    pixel_to_micrometer = files[0].scale[0]
    rolling_radius = int(0.5 * (25 * pixel_to_micrometer) / 2)
    stacks = []

    for f, frame_count in zip(files, frames):
        for t in range(frame_count):
            z_stack = np.average([i for i in f.get_iter_z(c=0, t=t)], axis=0)
            projection = z_stack - rolling_ball(z_stack, radius=rolling_radius)
            stacks.append(projection)

    images = []
    cmap = plt.cm.magma
    scale_bar_length_pixels = int(20.0 * pixel_to_micrometer)


    for idx, projection in enumerate(stacks):
        elapsed = total_seconds + idx * (float(1/files[0].scale[-1]))
        h, m, s = int(elapsed // 3600), int((elapsed % 3600) // 60), int(elapsed % 60)
        norm = Normalize(vmin=projection.min(), vmax=projection.max())
        img = Image.fromarray((cmap(norm(projection)) * 255).astype(np.uint8))
        draw = ImageDraw.Draw(img)
        try:
            font = ImageFont.truetype("Roboto-Light.ttf", 60)
        except:
            font = ImageFont.load_default()
        draw.text((20, 20), f"{h:02}:{m:02}:{s:02}", fill="white", font=font)
        draw.rectangle([(10, img.height - 40), (10 + scale_bar_length_pixels, img.height - 25)], fill="white")
        images.append(img)

    output_folder = os.path.splitext(leica_file)[0] + "_output"
    os.makedirs(output_folder, exist_ok=True)
    os.chdir(output_folder)
    images[0].save(f"{output_folder}_magma.gif", save_all=True, append_images=images[1:], duration=120, loop=0)
    os.chdir(original_dir)

    t_column = np.linspace(total_seconds, total_seconds + len(images)*(float(files[0].scale[-1])* float(files[0].nt)), len(images))
    return stacks, t_column, images

# === Image Processing: Colocalization ===
def process_lif_colocalization(
    leica_file,
    n_samples,
    font_path="Roboto-Light.ttf",
    scale_bar_length_micrometers=20.0,
    min_droplet_size=None,
    return_gif=True
):
    




    """
    
    
    
    1.	Setup & output folder creation.
	2.	Loop through frames:
	•	Extract channel 0 & 2 z-stacks
	•	Rolling-ball background subtraction
	•	Normalize channels (0→G, 2→R)
	•	Build RGB composite (R,G,B) = (Ch2, Ch0, 0)
	•	Annotate with time + scale bar
	•	Compute Pearson correlation coefficient
	•	Segment droplets (Otsu threshold → label regions)
	•	Measure droplet intensities (median per region)
	3.	Save outputs:
	•	Animated GIF with merged frames
	•	*_droplets.csv with droplet intensity data
	•	*_pearson.csv with Pearson correlation vs. time
	•	Individual RGB images in /RGB_images folder

    Returns (dictionary)
	•	"rgb": list of annotated RGB images (PIL)
	•	"pearson": list of Pearson correlation coefficients
	•	"droplet_dataframe": pandas.DataFrame with droplet intensities
	•	"droplet_count": number of detected droplets per frame
	•	"time_column": elapsed time values
	•	"output_folder": path where results are saved
    
    
    """
    # === 1. Setup ===
    original_dir = os.getcwd()
    base_name = os.path.splitext(os.path.basename(leica_file))[0]
    output_folder = os.path.join(original_dir, f"{base_name}_output")
    os.makedirs(output_folder, exist_ok=True)

    # Parse time from filename
    match = re.search(r'(\d+)min(\d+)sec', leica_file)
    t_focus = int(match.group(1)) * 60 + int(match.group(2)) if match else 0

    # Load file
    file = LifFile(os.path.join(original_dir, leica_file))
    files = [file.get_image(i) for i in range(n_samples)]
    pixel_to_micrometer = files[0].scale[0]
    rolling_radius = int(0.5 * (25 * pixel_to_micrometer) / 2)  # for max 20 µm droplets

    # Containers
    rgb_images = []
    pearson_coeffs = []
    droplet_data = []
    droplet_counts = []
    time_column = []
    labeled_images = []

    # === 2. Loop over files and frames ===
    for f in files:
        for t in range(f.nt):
            z0 = np.average([img for img in f.get_iter_z(c=0, t=t)], axis=0)
            z2 = np.average([img for img in f.get_iter_z(c=2, t=t)], axis=0)

            z0 -= rolling_ball(z0, radius=rolling_radius)
            z2 -= rolling_ball(z2, radius=rolling_radius)

            # Normalize channels
            norm0 = Normalize(vmin=z0.min(), vmax=z0.max())(z0)  # Ch0 = green
            norm2 = Normalize(vmin=z2.min(), vmax=z2.max())(z2)  # Ch2 = red

            # Create RGB image: [R, G, B] = [Ch2, Ch0, 0]
            rgb = np.stack([norm2, norm0, np.zeros_like(norm0)], axis=-1)
            rgb_uint8 = (rgb * 255).astype(np.uint8)
            img = Image.fromarray(rgb_uint8)

            # Annotate image
            draw = ImageDraw.Draw(img)
            try:
                font = ImageFont.truetype(font_path, 60)
            except OSError:
                print(f"Font '{font_path}' not found. Using default font.")
                font = ImageFont.load_default()
            
            elapsed = t_focus + len(rgb_images) * (float(1/files[0].scale[-1]))
            time_str = f"{int(elapsed // 3600):02}:{int((elapsed % 3600) // 60):02}:{int(elapsed % 60):02}"
            draw.text((20, 20), time_str, fill="white", font=font)

            scale_bar_length_px = int(scale_bar_length_micrometers * pixel_to_micrometer)
            draw.rectangle([(10, img.height - 40), (10 + scale_bar_length_px, img.height - 25)], fill="white")

            rgb_images.append(img)
            time_column.append(elapsed)
            pearson_coeffs.append(pearsonr(z0.flatten(), z2.flatten())[0])



            binary = z2 > threshold_otsu(z2)
            labeled = label(binary)
            labeled_images.append(labeled)

            for region in regionprops(labeled):
                coords = tuple(region.coords.T)
                mean_ch0 = np.median(z0[coords])
                mean_ch1 = np.median(z2[coords])
                droplet_data.append((mean_ch0, mean_ch1, elapsed / 60))

            droplet_counts.append(len(droplet_data))

    # === 3. Save output ===
    if return_gif and rgb_images:
        gif_path = os.path.join(output_folder, f"{base_name}_merged.gif")
        rgb_images[0].save(
            gif_path,
            save_all=True,
            append_images=rgb_images[1:],
            duration=120,
            loop=0,
            dpi=(300, 300)
        )

    df = pd.DataFrame(droplet_data, columns=["Ch0", "Ch1", "min"])
    df.to_csv(os.path.join(output_folder, f"{base_name}_droplets.csv"), index=False)



    p = pd.DataFrame({
        "time": time_column,
        "pearson": pearson_coeffs})
    

    p.to_csv(os.path.join(output_folder, f"{base_name}_pearson.csv"), index=False)
    
    # === Save individual RGB images ===
    rgb_folder = os.path.join(output_folder, "RGB_images")
    os.makedirs(rgb_folder, exist_ok=True)

    for idx, img in enumerate(rgb_images):
        elapsed = time_column[idx]
        h, m, s = int(elapsed // 3600), int((elapsed % 3600) // 60), int(elapsed % 60)
        timestamp = f"{h:02}h{m:02}m{s:02}s"
        img.save(os.path.join(rgb_folder, f"{base_name}_{timestamp}.png"), dpi=(300, 300))
    
    return {
        "rgb": rgb_images,
        "pearson": pearson_coeffs,
        "droplet_dataframe": df,
        "droplet_count": droplet_counts,
        "time_column": time_column,
        "output_folder": output_folder
    }






# DATA

## Output Files

For each .lif file, the script generates:

	*_output/ → main output folder

	*_magma.gif / *_merged.gif → animated timelapse

	*_droplets.csv → droplet data (median intensities, time)

	*_pearson.csv → Pearson correlation coefficients
	
	RGB_images/ → folder with per-frame annotated PNG

In [ ]:

for i in sorted(glob.glob("*.lif")):
    # Process max projection and save GIF
    stacks, t_column, images = process_multiple_file_kinetics_merge_max(i, n_samples=1)
    print(f"Processed kinetics file: {i}")

    # Process colocalization and droplet extraction
    results = process_lif_colocalization(i, n_samples=1, return_gif=True)
    print(f"Processed colocalization file: {i}")